In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier


In [2]:
data = pd.read_csv("..\dataset\\spam_ham_dataset.csv")

X = data['text']
y = data['label_num']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_seq_length = max([len(seq) for seq in X_train_seq])
X_train_pad = pad_sequences(X_train_seq, maxlen=max_seq_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_seq_length, padding='post')

In [4]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_seq_length))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=20, batch_size=16)

Epoch 1/20
259/259 [==============================] - 31s 115ms/step - loss: 0.6095 - accuracy: 0.7065 - val_loss: 0.5926 - val_accuracy: 0.7169
Epoch 2/20
259/259 [==============================] - 28s 108ms/step - loss: 0.5929 - accuracy: 0.7084 - val_loss: 0.5835 - val_accuracy: 0.7169
Epoch 3/20
259/259 [==============================] - 27s 104ms/step - loss: 0.5494 - accuracy: 0.7181 - val_loss: 0.4732 - val_accuracy: 0.7440
Epoch 4/20
259/259 [==============================] - 27s 104ms/step - loss: 0.3922 - accuracy: 0.8175 - val_loss: 0.3022 - val_accuracy: 0.8493
Epoch 5/20
259/259 [==============================] - 27s 103ms/step - loss: 0.2491 - accuracy: 0.9231 - val_loss: 0.2364 - val_accuracy: 0.9295
Epoch 6/20
259/259 [==============================] - 27s 104ms/step - loss: 0.1757 - accuracy: 0.9463 - val_loss: 0.1483 - val_accuracy: 0.9807
Epoch 7/20
259/259 [==============================] - 27s 103ms/step - loss: 0.1280 - accuracy: 0.9654 - val_loss: 0.1600 - val_ac

In [5]:
model.evaluate(X_test_pad, y_test)

33/33 [==============================] - 1s 33ms/step - loss: 0.0555 - accuracy: 0.9816


[0.0554976649582386, 0.981642484664917]

In [6]:
y_pred_prob = model.predict(X_test_pad)
y_pred = np.round(y_pred_prob).astype(int)
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()

print("confusion matrix:")
print(conf_matrix)
print("True Positives (TP) :", tp)
print("False Positives (TN) :", tn)

33/33 [==============================] - 6s 31ms/step
confusion matrix:
[[734   8]
 [ 11 282]]
True Positives (TP) : 282
False Positives (TN) : 734


In [7]:
model.save("..\\models\\mail.keras")

In [9]:
from keras.models import load_model
model1 = load_model("..\\models\\mail.keras")
model1.evaluate(X_test_pad, y_test)

33/33 [==============================] - 1s 28ms/step - loss: 0.0555 - accuracy: 0.9816


[0.0554976649582386, 0.981642484664917]